In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import re
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, plot_roc_curve, balanced_accuracy_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
import xgboost
import lightgbm
from sklearn import linear_model, tree, ensemble
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings(action='ignore')
pd.options.display.max_columns = None
import math

In [2]:
train=pd.read_csv('phase2_train.csv', index_col=0)
val=pd.read_csv('phase2_test.csv', index_col=0)

In [3]:
train

,User_ID,Gender,Age,MBTI,Q_number,Answer
0,1,1,30,INFP,1,아니다. 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의 ...
1,1,1,30,INFP,2,중립. 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하며...
2,1,1,30,INFP,3,그렇다. 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 영...
3,1,1,30,INFP,4,중립. 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고 ...
4,1,1,30,INFP,5,아니다. 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대로...
...,...,...,...,...,...,...
18715,480,0,30,ISTP,56,그렇다. 거래처에 가격 조정 때문에 3군데를 가야 하는 상황이었는데 이야기 잘 통하...
18716,480,0,30,ISTP,57,아니다. 상대방과 논쟁을 불러드릴 주제에는 관심이 없습니다 괜히 싸움을 일으키기 싫습니다
18717,480,0,30,ISTP,58,아니다. 나에게 온 기회를 포기할 수 없다 양보를 하게 되면 나에게 기회는 없어지니깐
18718,480,0,30,ISTP,59,아니다. 마감 기한이 정해지면 그 일을 끝날 때까지 늦게까지 일을 하고 퇴근하곤 합니다


In [4]:
val

,User_ID,Gender,Age,Q_number,Answer
0,241,0,30,1,아니다. 친구를 만들 상황에 새로운 친구를 만듭니다. 의도적으로나 꼭 주기적으로 새...
1,241,0,30,2,그렇다. 일상이 무료한 걸 못 참는 편. 요즘은 리본으로 뜨개질해서 가방 만드는 걸...
2,241,0,30,3,아니다. 다른 사람이 울고 있는 모습을 본다고 해도 눈물이 나지는 않습니다. 그 심...
3,241,0,30,4,그렇다. 진행하고 있는 일이 계획단계일 때 이미 대비책을 함께 세우는 편. 잘못된 ...
4,241,0,30,5,"그렇다. 최근 프로젝트 진행 중 문제가 발생, 마감기한이 얼마 남지 않았고, 주변 ..."
...,...,...,...,...,...
7195,453,0,20,56,아니다. 저는 절차대로 일을 완수하는 편이 아닙니다. 이유는 절차적으로 일을 완수하...
7196,453,0,20,57,아니다. 저는 논쟁을 불러일으킬 주제에 관심이 없습니다. 이유는 논쟁을 불러일으킬 ...
7197,453,0,20,58,아니다. 저는 다른 사람에게 더 필요한 기회라고 생각되면 기회를 포기할 수 없습니다...
7198,453,0,20,59,아니다. 저는 마감 기한을 지키기 쉽습니다. 이유는 주어진 기한 안에 문제를 해결하...


In [5]:
def col(df):
    mapping = {'I': 0, 'E': 1, 'S': 0, 'N': 1, 'T': 0, 'F': 1, 'J': 0, 'P': 1}

    df['I/E'] = df['MBTI'].apply(lambda x: mapping[x[0]])
    df['S/N'] = df['MBTI'].apply(lambda x: mapping[x[1]])
    df['T/F'] = df['MBTI'].apply(lambda x: mapping[x[2]])
    df['J/P'] = df['MBTI'].apply(lambda x: mapping[x[3]])
    return df

train=col(train)
train

,User_ID,Gender,Age,MBTI,Q_number,Answer,I/E,S/N,T/F,J/P
0,1,1,30,INFP,1,아니다. 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의 ...,0,1,1,1
1,1,1,30,INFP,2,중립. 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하며...,0,1,1,1
2,1,1,30,INFP,3,그렇다. 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 영...,0,1,1,1
3,1,1,30,INFP,4,중립. 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고 ...,0,1,1,1
4,1,1,30,INFP,5,아니다. 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대로...,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
18715,480,0,30,ISTP,56,그렇다. 거래처에 가격 조정 때문에 3군데를 가야 하는 상황이었는데 이야기 잘 통하...,0,0,0,1
18716,480,0,30,ISTP,57,아니다. 상대방과 논쟁을 불러드릴 주제에는 관심이 없습니다 괜히 싸움을 일으키기 싫습니다,0,0,0,1
18717,480,0,30,ISTP,58,아니다. 나에게 온 기회를 포기할 수 없다 양보를 하게 되면 나에게 기회는 없어지니깐,0,0,0,1
18718,480,0,30,ISTP,59,아니다. 마감 기한이 정해지면 그 일을 끝날 때까지 늦게까지 일을 하고 퇴근하곤 합니다,0,0,0,1


In [6]:
train['Answer'] = train['Answer'].str.split('.').str[0]
val['Answer'] = val['Answer'].str.split('.').str[0]

In [7]:
train

,User_ID,Gender,Age,MBTI,Q_number,Answer,I/E,S/N,T/F,J/P
0,1,1,30,INFP,1,아니다,0,1,1,1
1,1,1,30,INFP,2,중립,0,1,1,1
2,1,1,30,INFP,3,그렇다,0,1,1,1
3,1,1,30,INFP,4,중립,0,1,1,1
4,1,1,30,INFP,5,아니다,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
18715,480,0,30,ISTP,56,그렇다,0,0,0,1
18716,480,0,30,ISTP,57,아니다,0,0,0,1
18717,480,0,30,ISTP,58,아니다,0,0,0,1
18718,480,0,30,ISTP,59,아니다,0,0,0,1


In [8]:
val['I/E']=np.nan
val['S/N']=np.nan
val['T/F']=np.nan
val['J/P']=np.nan
val

,User_ID,Gender,Age,Q_number,Answer,I/E,S/N,T/F,J/P
0,241,0,30,1,아니다,NaN,NaN,NaN,NaN
1,241,0,30,2,그렇다,NaN,NaN,NaN,NaN
2,241,0,30,3,아니다,NaN,NaN,NaN,NaN
3,241,0,30,4,그렇다,NaN,NaN,NaN,NaN
4,241,0,30,5,그렇다,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7195,453,0,20,56,아니다,NaN,NaN,NaN,NaN
7196,453,0,20,57,아니다,NaN,NaN,NaN,NaN
7197,453,0,20,58,아니다,NaN,NaN,NaN,NaN
7198,453,0,20,59,아니다,NaN,NaN,NaN,NaN


In [9]:
train=train.drop(['User_ID', 'MBTI'], axis=1)
val=val.drop('User_ID', axis=1)
train

,Gender,Age,Q_number,Answer,I/E,S/N,T/F,J/P
0,1,30,1,아니다,0,1,1,1
1,1,30,2,중립,0,1,1,1
2,1,30,3,그렇다,0,1,1,1
3,1,30,4,중립,0,1,1,1
4,1,30,5,아니다,0,1,1,1
...,...,...,...,...,...,...,...,...
18715,0,30,56,그렇다,0,0,0,1
18716,0,30,57,아니다,0,0,0,1
18717,0,30,58,아니다,0,0,0,1
18718,0,30,59,아니다,0,0,0,1


In [10]:
def dummy_creation(df, columns_list):
    df_dummies = []
    for col in columns_list:
        df_dummies.append(pd.get_dummies(df[col], prefix = col, prefix_sep = ':'))
    df_dummies = pd.concat(df_dummies, axis = 1)
    df = pd.concat([df, df_dummies], axis = 1)
    return df

In [11]:
train['Answer'] = train['Answer'].str.replace(' ', '')
train['Answer'] = train['Answer'].str.replace(',', '')

val['Answer'] = val['Answer'].str.replace(' ', '')
val['Answer'] = val['Answer'].str.replace(',', '')

train['Answer'] = train['Answer'].str[0]
val['Answer'] = val['Answer'].str[0]

train['Answer'] = train['Answer'].replace('어', '아')

In [12]:
train['Answer'].value_counts()

그    7758
아    7489
중    3467
Name: Answer, dtype: int64

In [13]:
val['Answer'].value_counts()

그    3014
아    2988
중    1198
Name: Answer, dtype: int64

In [14]:
train

,Gender,Age,Q_number,Answer,I/E,S/N,T/F,J/P
0,1,30,1,아,0,1,1,1
1,1,30,2,중,0,1,1,1
2,1,30,3,그,0,1,1,1
3,1,30,4,중,0,1,1,1
4,1,30,5,아,0,1,1,1
...,...,...,...,...,...,...,...,...
18715,0,30,56,그,0,0,0,1
18716,0,30,57,아,0,0,0,1
18717,0,30,58,아,0,0,0,1
18718,0,30,59,아,0,0,0,1


In [15]:
val

,Gender,Age,Q_number,Answer,I/E,S/N,T/F,J/P
0,0,30,1,아,NaN,NaN,NaN,NaN
1,0,30,2,그,NaN,NaN,NaN,NaN
2,0,30,3,아,NaN,NaN,NaN,NaN
3,0,30,4,그,NaN,NaN,NaN,NaN
4,0,30,5,그,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
7195,0,20,56,아,NaN,NaN,NaN,NaN
7196,0,20,57,아,NaN,NaN,NaN,NaN
7197,0,20,58,아,NaN,NaN,NaN,NaN
7198,0,20,59,아,NaN,NaN,NaN,NaN


In [16]:
train = dummy_creation(train, ['Gender', 'Age', 'Q_number', 'Answer'])
val = dummy_creation(val, ['Gender', 'Age', 'Q_number', 'Answer'])

In [17]:
train

,Gender,Age,Q_number,Answer,I/E,S/N,T/F,J/P,Gender:0,Gender:1,Age:20,Age:30,Age:40,Q_number:1,Q_number:2,Q_number:3,Q_number:4,Q_number:5,Q_number:6,Q_number:7,Q_number:8,Q_number:9,Q_number:10,Q_number:11,Q_number:12,Q_number:13,Q_number:14,Q_number:15,Q_number:16,Q_number:17,Q_number:18,Q_number:19,Q_number:20,Q_number:21,Q_number:22,Q_number:23,Q_number:24,Q_number:25,Q_number:26,Q_number:27,Q_number:28,Q_number:29,Q_number:30,Q_number:31,Q_number:32,Q_number:33,Q_number:34,Q_number:35,Q_number:36,Q_number:37,Q_number:38,Q_number:39,Q_number:40,Q_number:41,Q_number:42,Q_number:43,Q_number:44,Q_number:45,Q_number:46,Q_number:47,Q_number:48,Q_number:49,Q_number:50,Q_number:51,Q_number:52,Q_number:53,Q_number:54,Q_number:55,Q_number:56,Q_number:57,Q_number:58,Q_number:59,Q_number:60,Answer:그,Answer:아,Answer:중
0,1,30,1,아,0,1,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,30,2,중,0,1,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,30,3,그,0,1,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,30,4,중,0,1,1,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,30,5,아,0,1,1,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18715,0,30,56,그,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
18716,0,30,57,아,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
18717,0,30,58,아,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
18718,0,30,59,아,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [18]:
val

,Gender,Age,Q_number,Answer,I/E,S/N,T/F,J/P,Gender:0,Gender:1,Age:20,Age:30,Age:40,Q_number:1,Q_number:2,Q_number:3,Q_number:4,Q_number:5,Q_number:6,Q_number:7,Q_number:8,Q_number:9,Q_number:10,Q_number:11,Q_number:12,Q_number:13,Q_number:14,Q_number:15,Q_number:16,Q_number:17,Q_number:18,Q_number:19,Q_number:20,Q_number:21,Q_number:22,Q_number:23,Q_number:24,Q_number:25,Q_number:26,Q_number:27,Q_number:28,Q_number:29,Q_number:30,Q_number:31,Q_number:32,Q_number:33,Q_number:34,Q_number:35,Q_number:36,Q_number:37,Q_number:38,Q_number:39,Q_number:40,Q_number:41,Q_number:42,Q_number:43,Q_number:44,Q_number:45,Q_number:46,Q_number:47,Q_number:48,Q_number:49,Q_number:50,Q_number:51,Q_number:52,Q_number:53,Q_number:54,Q_number:55,Q_number:56,Q_number:57,Q_number:58,Q_number:59,Q_number:60,Answer:그,Answer:아,Answer:중
0,0,30,1,아,NaN,NaN,NaN,NaN,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,30,2,그,NaN,NaN,NaN,NaN,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,30,3,아,NaN,NaN,NaN,NaN,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,30,4,그,NaN,NaN,NaN,NaN,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,30,5,그,NaN,NaN,NaN,NaN,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,0,20,56,아,NaN,NaN,NaN,NaN,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
7196,0,20,57,아,NaN,NaN,NaN,NaN,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
7197,0,20,58,아,NaN,NaN,NaN,NaN,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
7198,0,20,59,아,NaN,NaN,NaN,NaN,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [19]:
Y_train1=train['I/E']
Y_train2=train['S/N']
Y_train3=train['T/F']
Y_train4=train['J/P']

Y_val1=val['I/E']
Y_val2=val['S/N']
Y_val3=val['T/F']
Y_val4=val['J/P']

train=train.drop(columns=['Gender', 'Age', 'Q_number', 'Answer', 'I/E', 'S/N', 'T/F', 'J/P'])
val=val.drop(columns=['Gender', 'Age', 'Q_number', 'Answer', 'I/E', 'S/N', 'T/F', 'J/P'])

In [20]:
train

,Gender:0,Gender:1,Age:20,Age:30,Age:40,Q_number:1,Q_number:2,Q_number:3,Q_number:4,Q_number:5,Q_number:6,Q_number:7,Q_number:8,Q_number:9,Q_number:10,Q_number:11,Q_number:12,Q_number:13,Q_number:14,Q_number:15,Q_number:16,Q_number:17,Q_number:18,Q_number:19,Q_number:20,Q_number:21,Q_number:22,Q_number:23,Q_number:24,Q_number:25,Q_number:26,Q_number:27,Q_number:28,Q_number:29,Q_number:30,Q_number:31,Q_number:32,Q_number:33,Q_number:34,Q_number:35,Q_number:36,Q_number:37,Q_number:38,Q_number:39,Q_number:40,Q_number:41,Q_number:42,Q_number:43,Q_number:44,Q_number:45,Q_number:46,Q_number:47,Q_number:48,Q_number:49,Q_number:50,Q_number:51,Q_number:52,Q_number:53,Q_number:54,Q_number:55,Q_number:56,Q_number:57,Q_number:58,Q_number:59,Q_number:60,Answer:그,Answer:아,Answer:중
0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18715,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
18716,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
18717,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
18718,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [21]:
def model_perf(model, X, Y, prefix=''):
    p_hat = model.predict_proba(X)[:, 1]
    Y = Y.astype(int)
    auc = roc_auc_score(Y, p_hat)
    print (f'{prefix} AUROC score = {auc:.3f}')

# note : cross validation is not included because of computation time
def model_fit_perf(model, X_all, Y_all, prefix=''):
    X_trn, X_val, Y_trn, Y_val = train_test_split(X_all, Y_all, test_size=0.2, random_state=4)
    model.fit(X_trn, Y_trn)
    print (prefix)
    model_perf(model, X_trn, Y_trn, '  Training  ')
    model_perf(model, X_val, Y_val, '  Validation')

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(train, Y_train1, test_size = 0.2, random_state = 42, stratify = Y_train1)




model1 = CatBoostClassifier(iterations=2000,
                           depth = 6,
                           learning_rate = 0.1,
                          #  max_leaves = 31,
                           #min_data_in_leaf = 500,
                           l2_leaf_reg = 3*4, # def helped
                           n_estimators = None,
                            # random_strength = ,
                          #  rsm = 1.0,
                           loss_function='Logloss',
                           eval_metric = "AUC",
                          #  early_stopping_rounds = 50,
                           early_stopping_rounds = 100,

                          #  metric = "aucroc",
                           verbose=True)


model1.fit( 
    X_train, Y_train,
          eval_set = (X_test, Y_test),
          verbose=1,)

model_perf(model1, X_train, Y_train, '  Training  ')
model_perf(model1, X_test, Y_test, '  Validation')
#model_perf(model, X_valid, Y_valid, '  Validation2')


#fname = f"catboost_model_nokib"
#model1.save_model(fname,
           #format="cbm",
           #export_parameters=None,
           #pool=None)

#model1 = CatBoostClassifier().load_model(fname, format='cbm')


0:	test: 0.6214104	best: 0.6214104 (0)	total: 183ms	remaining: 6m 6s
1:	test: 0.6214104	best: 0.6214104 (0)	total: 191ms	remaining: 3m 11s
2:	test: 0.6214104	best: 0.6214104 (0)	total: 199ms	remaining: 2m 12s
3:	test: 0.6205990	best: 0.6214104 (0)	total: 207ms	remaining: 1m 43s
4:	test: 0.6225745	best: 0.6225745 (4)	total: 217ms	remaining: 1m 26s
5:	test: 0.6215222	best: 0.6225745 (4)	total: 226ms	remaining: 1m 15s
6:	test: 0.6204340	best: 0.6225745 (4)	total: 236ms	remaining: 1m 7s
7:	test: 0.6221528	best: 0.6225745 (4)	total: 245ms	remaining: 1m 1s
8:	test: 0.6211929	best: 0.6225745 (4)	total: 255ms	remaining: 56.5s
9:	test: 0.6209545	best: 0.6225745 (4)	total: 262ms	remaining: 52.2s
10:	test: 0.6220800	best: 0.6225745 (4)	total: 272ms	remaining: 49.1s
11:	test: 0.6208701	best: 0.6225745 (4)	total: 282ms	remaining: 46.7s
12:	test: 0.6225486	best: 0.6225745 (4)	total: 291ms	remaining: 44.5s
13:	test: 0.6217923	best: 0.6225745 (4)	total: 298ms	remaining: 42.2s
14:	test: 0.6212304	best:

120:	test: 0.6385339	best: 0.6406052 (80)	total: 1.35s	remaining: 21s
121:	test: 0.6384699	best: 0.6406052 (80)	total: 1.36s	remaining: 21s
122:	test: 0.6385384	best: 0.6406052 (80)	total: 1.37s	remaining: 21s
123:	test: 0.6384976	best: 0.6406052 (80)	total: 1.38s	remaining: 20.9s
124:	test: 0.6386007	best: 0.6406052 (80)	total: 1.39s	remaining: 20.9s
125:	test: 0.6389373	best: 0.6406052 (80)	total: 1.4s	remaining: 20.9s
126:	test: 0.6389090	best: 0.6406052 (80)	total: 1.41s	remaining: 20.8s
127:	test: 0.6387882	best: 0.6406052 (80)	total: 1.42s	remaining: 20.8s
128:	test: 0.6384762	best: 0.6406052 (80)	total: 1.43s	remaining: 20.8s
129:	test: 0.6385670	best: 0.6406052 (80)	total: 1.44s	remaining: 20.7s
130:	test: 0.6384768	best: 0.6406052 (80)	total: 1.45s	remaining: 20.7s
131:	test: 0.6383406	best: 0.6406052 (80)	total: 1.46s	remaining: 20.7s
132:	test: 0.6383685	best: 0.6406052 (80)	total: 1.47s	remaining: 20.6s
133:	test: 0.6384137	best: 0.6406052 (80)	total: 1.48s	remaining: 20.6s

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(train, Y_train2, test_size = 0.2, random_state = 42, stratify = Y_train2)




model2 = CatBoostClassifier(iterations=2000,
                           depth = 6,
                           learning_rate = 0.1,
                          #  max_leaves = 31,
                           #min_data_in_leaf = 500,
                           l2_leaf_reg = 3*4, # def helped
                           n_estimators = None,
                            # random_strength = ,
                          #  rsm = 1.0,
                           loss_function='Logloss',
                           eval_metric = "AUC",
                          #  early_stopping_rounds = 50,
                           early_stopping_rounds = 100,

                          #  metric = "aucroc",
                           verbose=True)

model2.fit( 
    X_train, Y_train,
          eval_set = (X_test, Y_test),
          verbose=1,)

model_perf(model2, X_train, Y_train, '  Training  ')
model_perf(model2, X_test, Y_test, '  Validation')

0:	test: 0.6037717	best: 0.6037717 (0)	total: 8.76ms	remaining: 17.5s
1:	test: 0.6037717	best: 0.6037717 (0)	total: 15.5ms	remaining: 15.5s
2:	test: 0.6032500	best: 0.6037717 (0)	total: 24.4ms	remaining: 16.3s
3:	test: 0.6043266	best: 0.6043266 (3)	total: 31.9ms	remaining: 15.9s
4:	test: 0.6038969	best: 0.6043266 (3)	total: 40.2ms	remaining: 16.1s
5:	test: 0.6028793	best: 0.6043266 (3)	total: 48.5ms	remaining: 16.1s
6:	test: 0.6035491	best: 0.6043266 (3)	total: 58.2ms	remaining: 16.6s
7:	test: 0.6031226	best: 0.6043266 (3)	total: 69.6ms	remaining: 17.3s
8:	test: 0.6024780	best: 0.6043266 (3)	total: 80.8ms	remaining: 17.9s
9:	test: 0.6034693	best: 0.6043266 (3)	total: 91.2ms	remaining: 18.2s
10:	test: 0.6034373	best: 0.6043266 (3)	total: 102ms	remaining: 18.5s
11:	test: 0.6050657	best: 0.6050657 (11)	total: 112ms	remaining: 18.6s
12:	test: 0.6048846	best: 0.6050657 (11)	total: 123ms	remaining: 18.8s
13:	test: 0.6043190	best: 0.6050657 (11)	total: 134ms	remaining: 19.1s
14:	test: 0.60329

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(train, Y_train3, test_size = 0.2, random_state = 42, stratify = Y_train3)




model3 = CatBoostClassifier(iterations=2000,
                           depth = 6,
                           learning_rate = 0.1,
                          #  max_leaves = 31,
                           #min_data_in_leaf = 500,
                           l2_leaf_reg = 3*4, # def helped
                           n_estimators = None,
                            # random_strength = ,
                          #  rsm = 1.0,
                           loss_function='Logloss',
                           eval_metric = "AUC",
                          #  early_stopping_rounds = 50,
                           early_stopping_rounds = 100,

                          #  metric = "aucroc",
                           verbose=True)

model3.fit( 
    X_train, Y_train,
          eval_set = (X_test, Y_test),
          verbose=1,)

model_perf(model3, X_train, Y_train, '  Training  ')
model_perf(model3, X_test, Y_test, '  Validation')

0:	test: 0.5835615	best: 0.5835615 (0)	total: 9.08ms	remaining: 18.2s
1:	test: 0.5808826	best: 0.5835615 (0)	total: 18.8ms	remaining: 18.8s
2:	test: 0.5821389	best: 0.5835615 (0)	total: 25.3ms	remaining: 16.9s
3:	test: 0.5808269	best: 0.5835615 (0)	total: 34.8ms	remaining: 17.3s
4:	test: 0.5794290	best: 0.5835615 (0)	total: 44.8ms	remaining: 17.9s
5:	test: 0.5784141	best: 0.5835615 (0)	total: 55.9ms	remaining: 18.6s
6:	test: 0.5769448	best: 0.5835615 (0)	total: 64.7ms	remaining: 18.4s
7:	test: 0.5784086	best: 0.5835615 (0)	total: 73.5ms	remaining: 18.3s
8:	test: 0.5804384	best: 0.5835615 (0)	total: 84.5ms	remaining: 18.7s
9:	test: 0.5804384	best: 0.5835615 (0)	total: 90.2ms	remaining: 18s
10:	test: 0.5805361	best: 0.5835615 (0)	total: 99ms	remaining: 17.9s
11:	test: 0.5804147	best: 0.5835615 (0)	total: 108ms	remaining: 17.9s
12:	test: 0.5809752	best: 0.5835615 (0)	total: 117ms	remaining: 17.9s
13:	test: 0.5814210	best: 0.5835615 (0)	total: 128ms	remaining: 18.1s
14:	test: 0.5813601	bes

121:	test: 0.5811287	best: 0.5865714 (26)	total: 1.15s	remaining: 17.7s
122:	test: 0.5809245	best: 0.5865714 (26)	total: 1.16s	remaining: 17.7s
123:	test: 0.5806678	best: 0.5865714 (26)	total: 1.17s	remaining: 17.7s
124:	test: 0.5802084	best: 0.5865714 (26)	total: 1.18s	remaining: 17.7s
125:	test: 0.5804234	best: 0.5865714 (26)	total: 1.19s	remaining: 17.8s
126:	test: 0.5801214	best: 0.5865714 (26)	total: 1.2s	remaining: 17.7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.5865714393
bestIteration = 26

Shrink model to first 27 iterations.
  Training   AUROC score = 0.605
  Validation AUROC score = 0.587


In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(train, Y_train4, test_size = 0.2, random_state = 42, stratify = Y_train4)




model4 = CatBoostClassifier(iterations=2000,
                           depth = 6,
                           learning_rate = 0.1,
                          #  max_leaves = 31,
                           #min_data_in_leaf = 500,
                           l2_leaf_reg = 3*4, # def helped
                           n_estimators = None,
                            # random_strength = ,
                          #  rsm = 1.0,
                           loss_function='Logloss',
                           eval_metric = "AUC",
                          #  early_stopping_rounds = 50,
                           early_stopping_rounds = 100,

                          #  metric = "aucroc",
                           verbose=True)

model4.fit( 
    X_train, Y_train,
          eval_set = (X_test, Y_test),
          verbose=1,)

model_perf(model4, X_train, Y_train, '  Training  ')
model_perf(model4, X_test, Y_test, '  Validation')

0:	test: 0.5682411	best: 0.5682411 (0)	total: 8.82ms	remaining: 17.6s
1:	test: 0.5678609	best: 0.5682411 (0)	total: 14.5ms	remaining: 14.5s
2:	test: 0.5693434	best: 0.5693434 (2)	total: 23.3ms	remaining: 15.5s
3:	test: 0.5717262	best: 0.5717262 (3)	total: 33ms	remaining: 16.5s
4:	test: 0.5760186	best: 0.5760186 (4)	total: 41.6ms	remaining: 16.6s
5:	test: 0.5758893	best: 0.5760186 (4)	total: 51.9ms	remaining: 17.3s
6:	test: 0.5758728	best: 0.5760186 (4)	total: 61.4ms	remaining: 17.5s
7:	test: 0.5735056	best: 0.5760186 (4)	total: 71.8ms	remaining: 17.9s
8:	test: 0.5741164	best: 0.5760186 (4)	total: 82.6ms	remaining: 18.3s
9:	test: 0.5749024	best: 0.5760186 (4)	total: 91.9ms	remaining: 18.3s
10:	test: 0.5744695	best: 0.5760186 (4)	total: 97.5ms	remaining: 17.6s
11:	test: 0.5732017	best: 0.5760186 (4)	total: 106ms	remaining: 17.6s
12:	test: 0.5732020	best: 0.5760186 (4)	total: 112ms	remaining: 17.1s
13:	test: 0.5727862	best: 0.5760186 (4)	total: 121ms	remaining: 17.1s
14:	test: 0.5732444	b

120:	test: 0.5782694	best: 0.5838588 (56)	total: 1.16s	remaining: 18s
121:	test: 0.5782420	best: 0.5838588 (56)	total: 1.17s	remaining: 18s
122:	test: 0.5782591	best: 0.5838588 (56)	total: 1.18s	remaining: 18s
123:	test: 0.5779636	best: 0.5838588 (56)	total: 1.19s	remaining: 18s
124:	test: 0.5779122	best: 0.5838588 (56)	total: 1.2s	remaining: 18s
125:	test: 0.5777831	best: 0.5838588 (56)	total: 1.21s	remaining: 18s
126:	test: 0.5777097	best: 0.5838588 (56)	total: 1.22s	remaining: 18s
127:	test: 0.5779319	best: 0.5838588 (56)	total: 1.23s	remaining: 18s
128:	test: 0.5776611	best: 0.5838588 (56)	total: 1.24s	remaining: 18s
129:	test: 0.5777408	best: 0.5838588 (56)	total: 1.25s	remaining: 18s
130:	test: 0.5773533	best: 0.5838588 (56)	total: 1.26s	remaining: 18s
131:	test: 0.5771429	best: 0.5838588 (56)	total: 1.27s	remaining: 18s
132:	test: 0.5771486	best: 0.5838588 (56)	total: 1.29s	remaining: 18.1s
133:	test: 0.5770118	best: 0.5838588 (56)	total: 1.29s	remaining: 18s
134:	test: 0.576500

In [26]:
val

,Gender:0,Gender:1,Age:20,Age:30,Age:40,Q_number:1,Q_number:2,Q_number:3,Q_number:4,Q_number:5,Q_number:6,Q_number:7,Q_number:8,Q_number:9,Q_number:10,Q_number:11,Q_number:12,Q_number:13,Q_number:14,Q_number:15,Q_number:16,Q_number:17,Q_number:18,Q_number:19,Q_number:20,Q_number:21,Q_number:22,Q_number:23,Q_number:24,Q_number:25,Q_number:26,Q_number:27,Q_number:28,Q_number:29,Q_number:30,Q_number:31,Q_number:32,Q_number:33,Q_number:34,Q_number:35,Q_number:36,Q_number:37,Q_number:38,Q_number:39,Q_number:40,Q_number:41,Q_number:42,Q_number:43,Q_number:44,Q_number:45,Q_number:46,Q_number:47,Q_number:48,Q_number:49,Q_number:50,Q_number:51,Q_number:52,Q_number:53,Q_number:54,Q_number:55,Q_number:56,Q_number:57,Q_number:58,Q_number:59,Q_number:60,Answer:그,Answer:아,Answer:중
0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
7196,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
7197,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
7198,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [27]:
ie=np.array([sub_array[0] for sub_array in model1.predict_proba(val)])
sn=np.array([sub_array[0] for sub_array in model2.predict_proba(val)])
tf=np.array([sub_array[0] for sub_array in model3.predict_proba(val)])
jp=np.array([sub_array[0] for sub_array in model4.predict_proba(val)])

In [28]:
subm=pd.read_csv('phase2_test.csv', index_col=0)

In [29]:
subm['User_ID']=subm['User_ID']-240
subm

,User_ID,Gender,Age,Q_number,Answer
0,1,0,30,1,아니다. 친구를 만들 상황에 새로운 친구를 만듭니다. 의도적으로나 꼭 주기적으로 새...
1,1,0,30,2,그렇다. 일상이 무료한 걸 못 참는 편. 요즘은 리본으로 뜨개질해서 가방 만드는 걸...
2,1,0,30,3,아니다. 다른 사람이 울고 있는 모습을 본다고 해도 눈물이 나지는 않습니다. 그 심...
3,1,0,30,4,그렇다. 진행하고 있는 일이 계획단계일 때 이미 대비책을 함께 세우는 편. 잘못된 ...
4,1,0,30,5,"그렇다. 최근 프로젝트 진행 중 문제가 발생, 마감기한이 얼마 남지 않았고, 주변 ..."
...,...,...,...,...,...
7195,213,0,20,56,아니다. 저는 절차대로 일을 완수하는 편이 아닙니다. 이유는 절차적으로 일을 완수하...
7196,213,0,20,57,아니다. 저는 논쟁을 불러일으킬 주제에 관심이 없습니다. 이유는 논쟁을 불러일으킬 ...
7197,213,0,20,58,아니다. 저는 다른 사람에게 더 필요한 기회라고 생각되면 기회를 포기할 수 없습니다...
7198,213,0,20,59,아니다. 저는 마감 기한을 지키기 쉽습니다. 이유는 주어진 기한 안에 문제를 해결하...


In [30]:
sub=pd.DataFrame()
sub['User_ID']=subm['User_ID']
sub['I/E']=ie
sub['S/N']=sn
sub['T/F']=tf
sub['J/P']=jp

sub

,User_ID,I/E,S/N,T/F,J/P
0,1,0.596778,0.499284,0.582915,0.478357
1,1,0.447192,0.493674,0.568017,0.446666
2,1,0.468539,0.499284,0.597666,0.483680
3,1,0.444356,0.492009,0.582540,0.500831
4,1,0.422006,0.492009,0.570937,0.447378
...,...,...,...,...,...
7195,213,0.613394,0.586018,0.548187,0.365035
7196,213,0.658075,0.572803,0.555336,0.377967
7197,213,0.557132,0.572803,0.577489,0.365634
7198,213,0.617818,0.572803,0.575561,0.345607


In [31]:
sub[sub['User_ID']==45]

,User_ID,I/E,S/N,T/F,J/P
2520,45,0.376404,0.441963,0.620497,0.431381
2521,45,0.252847,0.434856,0.606293,0.442939
2522,45,0.254628,0.433682,0.541037,0.439715
2523,45,0.271555,0.441963,0.613118,0.332658
2524,45,0.288303,0.441963,0.618246,0.405090
2525,45,0.345781,0.433682,0.592238,0.435863
2526,45,0.239540,0.433682,0.590177,0.497392
2527,45,0.248569,0.433682,0.562201,0.434589
2528,45,0.251004,0.433682,0.593901,0.524315
2529,45,0.265802,0.441963,0.619887,0.394075


In [32]:
def custom_agg(x):
    pct_above = np.mean(x > 0.5)
    if pct_above > 0.9:
        return 1
    elif pct_above < 0.1:  # not 90% values are > 0.5 implies that 90% values are < 0.5
        return 0
    else:
        return x.mean()

sub.groupby('User_ID').agg(custom_agg).reset_index()

,User_ID,I/E,S/N,T/F,J/P
0,1,0.445563,0.000000,1.000000,0.473157
1,2,0.000000,1.000000,0.498974,0.000000
2,3,1.000000,0.000000,0.000000,1.000000
3,4,0.450278,0.000000,1.000000,0.475935
4,5,0.000000,0.492093,1.000000,0.470398
...,...,...,...,...,...
115,175,1.000000,1.000000,0.000000,1.000000
116,178,0.575778,1.000000,0.000000,1.000000
117,179,0.000000,0.000000,1.000000,0.000000
118,191,0.453188,0.495191,1.000000,0.470760


In [33]:
df_grouped = sub.groupby('User_ID').mean().reset_index()

df_grouped['I/E']=1-df_grouped['I/E']
df_grouped['S/N']=1-df_grouped['S/N']
df_grouped['T/F']=1-df_grouped['T/F']
df_grouped['J/P']=1-df_grouped['J/P']


df_grouped.to_csv('classic.csv', index=False)

In [34]:
df_grouped

,User_ID,I/E,S/N,T/F,J/P
0,1,0.554437,0.508744,0.424330,0.526843
1,2,0.574794,0.470684,0.501026,0.559329
2,3,0.334080,0.584869,0.587057,0.417020
3,4,0.549722,0.508609,0.428119,0.524065
4,5,0.557408,0.507907,0.430657,0.529602
...,...,...,...,...,...
115,175,0.404963,0.438713,0.576039,0.438261
116,178,0.424222,0.438505,0.573754,0.439182
117,179,0.704690,0.556012,0.420282,0.589328
118,191,0.546812,0.504809,0.423454,0.529240


In [35]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


df_grouped.iloc[:, -4:] = scaler.fit_transform(df_grouped.iloc[:, -4:])
df_grouped

,User_ID,I/E,S/N,T/F,J/P
0,1,0.588027,0.529072,0.147301,0.628698
1,2,0.638513,0.328194,0.540656,0.771548
2,3,0.041521,0.930855,0.981891,0.145774
3,4,0.576334,0.528358,0.166735,0.616482
4,5,0.595396,0.524653,0.179750,0.640832
...,...,...,...,...,...
115,175,0.217317,0.159454,0.925384,0.239176
116,178,0.265080,0.158358,0.913665,0.243224
117,179,0.960668,0.778552,0.126535,0.903465
118,191,0.569116,0.508304,0.142809,0.639240
